In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [3]:
# Define transformations (ResNet requires specific normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load dataset
full_train_data = datasets.ImageFolder(root="C://Users//HP//Desktop//NNDL_Project//train", transform=transform)
test_data = datasets.ImageFolder(root="C://Users//HP//Desktop//NNDL_Project//test", transform=transform)

In [5]:
# Train-validation split (80% train, 20% validation)
train_size = int(0.8 * len(full_train_data))
val_size = len(full_train_data) - train_size
train_data, val_data = random_split(full_train_data, [train_size, val_size])


In [6]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [7]:
import torch.nn as nn
from torchvision import models

# Load the pretrained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

# Modify the classifier to output 7 classes
model.classifier[1] = nn.Conv2d(512, 7, kernel_size=(1,1), stride=(1,1))

# Now the model is ready to be used for training on your emotion dataset


c:\Users\HP\Desktop\NNDL_Project\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\Desktop\NNDL_Project\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:05<00:00, 947kB/s] 


In [8]:
# Move model to GPU if available
model = model.to(device)

In [9]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
from tqdm import tqdm  # Import tqdm for progress bars


# # Load the model's state dict from epoch 5
# checkpoint_path = "resnet18_emotion_epoch_11.pth"
# model.load_state_dict(torch.load(checkpoint_path))




start_epoch = 0
num_epochs = 20  

# Training Loop from Epoch 6
for epoch in range(start_epoch, num_epochs):
    model.train()
    train_loss = 0
    
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}", total=len(train_loader)):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    # Validation Loop
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}", total=len(val_loader)):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate Accuracy, Precision, and Recall
    val_accuracy = 100 * correct / total
    val_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Accuracy: {val_accuracy:.2f}%, Val Precision: {val_precision:.2f}, Val Recall: {val_recall:.2f}")

    # Save model after each epoch
    torch.save(model.state_dict(), f"squeezenet1_1_motion_epoch_{epoch+1}.pth")

# Final Evaluation on Test Set
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating Test Set", total=len(test_loader)):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * correct / total
test_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
test_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)

print(f"Test Accuracy: {test_accuracy:.2f}%, Test Precision: {test_precision:.2f}, Test Recall: {test_recall:.2f}")


Validating Epoch 1: 100%|██████████| 180/180 [00:19<00:00,  9.23it/s]


Epoch 1, Train Loss: 1.8447, Val Loss: 1.8286, Val Accuracy: 21.33%, Val Precision: 0.35, Val Recall: 0.21


Validating Epoch 2: 100%|██████████| 180/180 [00:12<00:00, 14.98it/s]


Epoch 2, Train Loss: 1.7767, Val Loss: 1.7038, Val Accuracy: 30.48%, Val Precision: 0.41, Val Recall: 0.30


Validating Epoch 3: 100%|██████████| 180/180 [00:14<00:00, 12.02it/s]


...

Epoch 19, Train Loss: 0.7546, Val Loss: 1.2169, Val Accuracy: 57.92%, Val Precision: 0.57, Val Recall: 0.58


Validating Epoch 20: 100%|██████████| 180/180 [00:16<00:00, 11.23it/s]


Epoch 20, Train Loss: 0.7169, Val Loss: 1.2519, Val Accuracy: 58.76%, Val Precision: 0.61, Val Recall: 0.59


Evaluating Test Set: 100%|██████████| 225/225 [00:28<00:00,  8.02it/s]

Test Accuracy: 57.70%, Test Precision: 0.60, Test Recall: 0.58
